In [340]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler 
from scipy.signal import argrelextrema
from tqdm import tqdm
from sklearn.svm import SVR,NuSVR
from sklearn.model_selection import GridSearchCV
from multiprocessing import Pool
from scipy import stats
from joblib import Parallel, delayed
from sklearn.neural_network import MLPRegressor
from tsfresh.feature_extraction import feature_calculators

subset_size = [629145481, 
               100147179,
               18227196][0]

In [311]:
train=pd.read_csv("../input/train.csv",nrows=subset_size, dtype={"acoustic_data": np.int16, "time_to_failure": np.float32})

In [341]:
def exps_to_data(train):
    diff = train['time_to_failure'].diff().abs()
    diff[0] = 0.001
    exps_indices = np.array(diff[diff > 0.1].index)
    
    l = 0
    r = 150000
    jump = 35345
    while r < train.shape[0]:
        if all(np.logical_or(exps_indices <= l, exps_indices >= r)):
            if train.iloc[l: r, :]['time_to_failure'].values[-1]>0.5:
                yield train.iloc[l: r, :]
            l += jump
            r += jump
        else:
            l += 1
            r += 1
segments = sum(1 for i in exps_to_data(train))
print(segments)

454


In [327]:
col_names = ['mean','max','variance','min', 'stdev', 'q1', 'q5','q95', 'q99',
             'A0',
'fft0_mean','fft0_max','fft0_min','fft0_q1','fft0_q5','fft0_q25','fft0_q50','fft0_q75','fft0_q95','fft0_q99','fft0_std',
'fft1_mean','fft1_max','fft1_min','fft1_q1','fft1_q5','fft1_q25','fft1_q50','fft1_q75','fft1_q95','fft1_q99','fft1_std',
'fft2_mean','fft2_max','fft2_min','fft2_q1','fft2_q5','fft2_q25','fft2_q50','fft2_q75','fft2_q95','fft2_q99','fft2_std',
'fft3_mean','fft3_max','fft3_min','fft3_q1','fft3_q5','fft3_q25','fft3_q50','fft3_q75','fft3_q95','fft3_q99','fft3_std',
             'q05_rolling_std_100', 'num_peak_1', 'autocorrelation_1'
            ]    

def preprocess(seg, segment, X1, Y1=None):
        if not Y1 is None:
            Y1.loc[segment, 'time_to_failure'] = seg['time_to_failure'].values[-1]

        x = seg['acoustic_data'].values
        X1.loc[segment, 'mean'] = x.mean()
        X1.loc[segment, 'stdev'] = x.std()
        X1.loc[segment, 'variance'] = X1.loc[segment, 'stdev']**2
        X1.loc[segment, 'max'] = x.max()
        X1.loc[segment, 'min'] = x.min()
        X1.loc[segment][['q1','q5','q95','q99']] =  np.quantile(x, [0.01,0.05,0.95,0.99]).transpose()
        X1.loc[segment, 'q05_rolling_std_100'] = np.quantile(seg['acoustic_data'].rolling(100).std().dropna().values, 0.05)
        
        X1.loc[segment, 'autocorrelation_1'] = feature_calculators.autocorrelation(x, 1)
        X1.loc[segment, 'num_peak_1'] = feature_calculators.number_peaks(x, 1)
        
        fft = np.abs(np.fft.fft(x)[:11000])
        X1.loc[segment, 'A0'] = fft[0]
        fft_lines = [1500,2500,4100,7900,11000]
        for i in range(len(fft_lines)-1):
            cur = fft[fft_lines[i]:fft_lines[i+1]]
            X1.loc[segment,f'fft{i}_mean'] = cur.mean()
            X1.loc[segment,f'fft{i}_max'] = cur.max()
            X1.loc[segment,f'fft{i}_min'] = cur.min()
            X1.loc[segment,f'fft{i}_q1'] = np.quantile(cur, 0.5)
            X1.loc[segment,f'fft{i}_q5'] = np.quantile(cur, 0.5)
            X1.loc[segment,f'fft{i}_q25'] = np.quantile(cur, 0.5)
            X1.loc[segment,f'fft{i}_q50'] = np.quantile(cur, 0.5)
            X1.loc[segment,f'fft{i}_q75'] = np.quantile(cur, 0.5)
            X1.loc[segment,f'fft{i}_q95'] = np.quantile(cur, 0.5)
            X1.loc[segment,f'fft{i}_q99'] = np.quantile(cur, 0.5)
            X1.loc[segment,f'fft{i}_std'] = cur.std()
cnt_segments = sum([1 for _ in exps_to_data(train)])

In [328]:
X1 = pd.DataFrame(index=range(segments), dtype=np.float64, columns=col_names)
Y1 = pd.DataFrame(index=range(segments), dtype=np.float64, columns=['time_to_failure'])

_=[preprocess(seg, segment, X1, Y1) for segment, seg in tqdm(enumerate(exps_to_data(train)),total=cnt_segments)]


100%|██████████| 52/52 [00:02<00:00, 23.43it/s]

In [329]:
# segments = [x for x in exps_to_data(train)]
# X1 = pd.DataFrame(index=range(len(segments)), dtype=np.float64)
# for seg_id, seg in enumerate(segments):
#     x = seg['acoustic_data'].values
#     X1.loc[seg_id,'time_to_failure'] = seg['time_to_failure'].values[-1]
#     std_res = seg['acoustic_data'].rolling(1000).std().dropna().values
#     X1.loc[seg_id, 'q5'] = np.quantile(std_res, 0.05)



In [330]:
# sc=StandardScaler()
# X2 = pd.DataFrame(sc.fit_transform(X1), columns = X1.columns)
# plt.plot(X2)
# plt.show()

In [331]:
sc=StandardScaler()
sc.fit(X1)
scX = pd.DataFrame(sc.transform(X1), columns = X1.columns)

In [332]:
parameters = [{'hidden_layer_sizes': [(100,), (100,100,), (100,100,100,)], 'activation': ['logistic', 'tanh', 'relu']}]
model = GridSearchCV(MLPRegressor(), parameters, cv=5, scoring='neg_mean_absolute_error', n_jobs=4)
model.fit(scX, Y1.values.flatten())

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid=[{'hidden_layer_sizes': [(100,), (100, 100), (100, 100, 100, 100, 100)], 'activation': ['logistic', 'tanh', 'relu']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

In [333]:
model.cv_results_

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

{'mean_fit_time': array([0.63217359, 2.41135283, 3.61105242, 0.55295267, 1.82050338,
        3.374157  , 0.47811394, 1.48810306, 2.08771238]),
 'std_fit_time': array([0.10002107, 0.27053276, 0.17376128, 0.21799129, 0.23695676,
        1.3635432 , 0.06865193, 0.06488646, 0.5870199 ]),
 'mean_score_time': array([0.00264935, 0.00194688, 0.00208597, 0.00455799, 0.00316343,
        0.00210915, 0.00308304, 0.00165997, 0.00330353]),
 'std_score_time': array([1.10658628e-03, 1.71539502e-04, 1.12528255e-04, 5.79634793e-03,
        2.72444792e-03, 1.19738985e-04, 2.88030609e-03, 3.73977154e-05,
        2.70761684e-03]),
 'param_activation': masked_array(data=['logistic', 'logistic', 'logistic', 'tanh', 'tanh',
                    'tanh', 'relu', 'relu', 'relu'],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_hidden_layer_sizes': masked_array(data=[(100,), (100, 100), (100, 100, 10

In [334]:
sub=pd.read_csv("../input/sample_submission.csv",index_col='seg_id')
xtest=pd.DataFrame(columns=X1.columns,dtype=np.float64,index=sub.index)

In [335]:
_ = Parallel(n_jobs=20, require='sharedmem', verbose=0)(
    delayed(preprocess)(pd.read_csv('../input/test/' + seg_id + '.csv'), seg_id, xtest) for seg_id in tqdm(xtest.index))


 15%|█▌        | 396/2624 [00:19<01:48, 20.48it/s]


 30%|███       | 796/2624 [00:40<01:33, 19.45it/s]


 45%|████▌     | 1193/2624 [01:00<01:12, 19.87it/s]


 60%|█████▉    | 1573/2624 [01:19<00:51, 20.40it/s]


 75%|███████▌  | 1973/2624 [01:40<00:36, 17.83it/s]


 90%|█████████ | 2370/2624 [02:00<00:12, 20.26it/s]


100%|██████████| 2624/2624 [02:13<00:00, 19.71it/s]

In [336]:
sctestx = pd.DataFrame(sc.transform(xtest), columns = xtest.columns)

In [337]:
pred = model.predict(sctestx)
print(pred.shape)

(2624,)


In [338]:
sub['time_to_failure'] = pred
sub.head()

,time_to_failure
seg_id,
seg_00030f,5.524024
seg_0012b5,3.347173
seg_00184e,3.778955
seg_003339,9.521089
seg_0042cc,6.488661


In [339]:
sub.to_csv("submittedoutput.csv")